In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sb
sb.set()
plt.rcParams['figure.figsize'] = (12,9)
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import numpy.random as npr
import chainer as ch
import chainer.functions as F
import chainer.links as L
import chainer.training.extensions

In [547]:
from dataset import load_data
from vocab import Vocab
from word_vectors import get_pretrained_vectors

[autoreload of updater failed: Traceback (most recent call last):
  File "/Users/thomaseffland/.virtualenvs/research/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
NameError: name 'convert' is not defined
]


# Setup data

In [4]:
train_data, dev_data = load_data("data/multinli_0.9/", 
                                 matched=True, 
                                 genres=['government'],
                                 drop_confused=True, 
                                 lowercase=True)
print("{} training examples, {} dev examples".format(len(train_data), len(dev_data)))

77350 training examples, 1945 dev examples


In [5]:
vocab = Vocab(min_count=1)
vocab.add([ token for datum in train_data for token in datum['h'] ])
vocab.add([ token for datum in train_data for token in datum['p'] ])
vocab.add([ token for datum in dev_data for token in datum['h'] ])
vocab.add([ token for datum in dev_data for token in datum['p'] ])
vocab.drop_infrequent()

In [6]:
token_embeddings = get_pretrained_vectors(vocab, 'data/word_vectors/glove.6B.300d.txt', 
                                          normed=True, trim=False)

Pretrained coverage: 21967/27285 = 80.51%


In [7]:
class_vocab = Vocab([ datum['c'] for datum in train_data ],
                    min_count=0,
                    pad_token=None,
                    unk_token=None)

# Define Models

In [537]:
from monitor import monitor

# Instantiate models and run experiment

# What to track:
* Number of parameters
* Number of trainable parameters
* Start time
* Examples per second
* Updates per second
* Model score vs time
* Ratio of parameter values to updates (on average for each layer)
* Mean, standard dev, and histograms of param updates, and activations
* Learning rate  over time
* Do model exponentially decayed model weight averaging. Do a moving average.

In [522]:
from retain_grad import RetainGrad

In [551]:
from better_report import BetterLogReport

In [552]:
from activation_monitor import ActivationMonitorExtension
from backprop_monitor import BackpropMonitorExtension

In [561]:
from updater import VariableConverterUpdater
from evaluator import VariableConverterEvaluator
from converter import NLIBatchConverter

In [587]:
cbow = CBOW(token_embeddings)
c_model= MLP([1000, 600, 300, 3])
predictor = NLIPredictor(cbow, c_model)
loss_model = NLILossModel(predictor)


optimizer = RetainGrad(ch.optimizers.Adam)()
# optimizer = ch.optimizers.Adam()
optimizer.setup(loss_model)
# optimizer.add_hook(ForwardBackwardMonitorHook())

batch_size = 256
k = 256
train_iter = ch.iterators.SerialIterator(train_data[:k], batch_size, shuffle=True, repeat=True)
dev_iter = ch.iterators.SerialIterator(dev_data[:k], batch_size, shuffle=False, repeat=False)

In [658]:
import yaml

In [659]:
cat experiment_config.yaml

n_epoch: 100

adam:
    learning_rate: .001
    
data_setup:
    setup_file: "data_setup.py"
    setup_config:
        batch_size: 256
        max_examples: "all"
        data_dir: "data/multinli_0.9/"
        matched: True
        genres: [ 'government' ]
        drop_confused: True
        lowercase: True
        pretrained_vectors: "data/word_vectors/glove.6B.300d.txt"
        normalize_vectors: True
        

In [660]:
config = yaml.load(open('experiment_config.yaml'))

In [664]:
from chainer import Chain, functions as F, links as L

In [661]:
config['data_setup']


{'setup_config': {'batch_size': 256,
  'data_dir': 'data/multinli_0.9/',
  'drop_confused': True,
  'genres': ['government'],
  'lowercase': True,
  'matched': True,
  'max_examples': 'all',
  'normalize_vectors': True,
  'pretrained_vectors': 'data/word_vectors/glove.6B.300d.txt'},
 'setup_file': 'data_setup.py'}

In [663]:
import importlib

In [588]:
converter = NLIBatchConverter(vocab, class_vocab)
updater = VariableConverterUpdater(train_iter, optimizer, converter=converter)
evaluator = VariableConverterEvaluator(dev_iter, loss_model, converter=converter)
activation_monitor = ActivationMonitorExtension()
backprop_monitor = BackpropMonitorExtension(loss_model)
logger = BetterLogReport(trigger=(1,'iteration'))

trainer = ch.training.Trainer(updater, (100, 'epoch'), out='result_test')
trainer.extend(evaluator)
trainer.extend(activation_monitor)
trainer.extend(backprop_monitor)
trainer.extend(logger)
# trainer.extend(ch.training.extensions.LogReport(trigger=(1,'iteration'),
#                                                 postprocess=postprocess))
trainer.extend(ch.training.extensions.PrintReport([
    'epoch', 'main/loss', 'main/accuracy', 'validation/main/accuracy'],
    log_report=logger
))
# trainer.extend(ch.training.extensions.ProgressBar())
# trainer.extend(ch.training.extensions.snapshot(
#     trigger=ch.training.triggers.MaxValueTrigger('validation/main/accuracy', (1,'epoch'))
# ))

In [589]:
trainer.run()

epoch       main/loss   main/accuracy  validation/main/accuracy
1           1.15964     0.34375        0.324219                  
1           1.15964     0.34375        0.324219                  
2           2.54482     0.371094       0.339844                  
2           2.54482     0.371094       0.339844                  
3           2.23368     0.292969       0.3125                    
3           2.23368     0.292969       0.3125                    
4           1.23733     0.339844       0.34375                   
4           1.23733     0.339844       0.34375                   
5           1.18258     0.324219       0.335938                  
5           1.18258     0.324219       0.335938                  
6           1.25712     0.34375        0.335938                  
6           1.25712     0.34375        0.335938                  
7           1.21673     0.34375        0.335938                  
7           1.21673     0.34375        0.335938                  
8           

60          1.09915     0.347656       0.324219                  
60          1.09915     0.347656       0.324219                  
61          1.09545     0.351562       0.324219                  
61          1.09545     0.351562       0.324219                  
62          1.10299     0.382812       0.3125                    
62          1.10299     0.382812       0.3125                    
63          1.10246     0.347656       0.324219                  
63          1.10246     0.347656       0.324219                  
64          1.10151     0.339844       0.296875                  
64          1.10151     0.339844       0.296875                  
65          1.0961      0.398438       0.277344                  
65          1.0961      0.398438       0.277344                  
66          1.10099     0.332031       0.296875                  
66          1.10099     0.332031       0.296875                  
67          1.10473     0.324219       0.296875                  
67        

In [ ]:
ch.optimizers.Adam()

# TODO

* [X] Compute micro, macro, and class-wise f1s
* [X] Report training and validation metrics
* [X] Report activation histograms
* [X] Report activation means, variances
* [X] Report gradient _update_ histograms
* [X] Report gradient _update_ means, variances


* [ ] Checkpoint model and be able to restore model from checkpoint
* [ ] Visualize the training as it progresses
* [ ] Batch out multiple experiments
* [ ] Post output to results folder that is timestamped
* [ ] Output config to results folder
* [ ] 

Need to be able to:
* Run experiments but exchange architectures
* Reproduce all experiments
* Analyze results, diagnose optimization and data errors

In [593]:
main_loss = [l['main/accuracy'] for l in logger._log]

In [607]:
from IPython.display import HTML

In [633]:
css = """
h3 {
    color:grey;
}
"""

In [638]:
viz = """
d3.select('#main-viz')
    .html('d3 inserted this!')
"""

In [641]:
HTML(open('vis_test.html').read())

In [640]:
html_str = """
<style> {css} </style>
<div> <h3> This is a visualization </h3> </div>
<div id="main-viz"></div>
<script> {viz_js} </script>
"""
HTML(html_str.format(css=css, viz_js=viz))

[How to embed d3 in jupyter](http://blog.thedataincubator.com/2015/08/embedding-d3-in-an-ipython-notebook/)

In [606]:
%%javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min'
  }
});
var margin = {top: 20, right: 10, bottom: 20, left: 10};
var width = 960 - margin.left - margin.right,
    height = 500 - margin.top - margin.bottom;
var svg = d3.select("element").append("svg")
    .attr("width", width + margin.left + margin.right)
    .attr("height", height + margin.top + margin.bottom)
  .append("g")
    .attr("transform", "translate(" + margin.left + "," + margin.top + ")")
var x = d3.scale.linear()
    .range([0, width]);
var y = d3.scale.linear()
    .range([height, 0]);

<IPython.core.display.Javascript object>